In [1]:
import datetime
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('201901-202003.csv')

# holiday 컬럼 만들기

In [3]:
# 2019년, 2020년 주말

def print_whichday(year, month, day) :
    r = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    aday = datetime.date(year, month, day)
    bday = aday.weekday()
    
    return r[bday]



weekend2019 = []
weekend2020 = []

for month in range(1, 13) :
    day2019 = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    
    week2019 = []
    
    for day in range(1, day2019[month-1] + 1) :
        
        week2019.append(print_whichday(2019, month, day))
        
    weekend2019.append(week2019.count('토요일') + week2019.count('일요일'))

    
for month in range(1, 8) :
    day2020 = [31, 29, 31, 30, 31, 30, 31]
    
    week2020 = []
    for day in range(1, day2020[month-1] + 1) :
        
        week2020.append(print_whichday(2020, month, day))
        
    weekend2020.append(week2020.count('토요일') + week2020.count('일요일'))

In [4]:
# 각 달마다 주말 수
weekend = weekend2019 + weekend2020

# 201901부터 202003까지
yymm = list(data['REG_YYMM'].unique())

# 202004 ~ 202007 추가
yymm.extend([202004, 202005, 202006, 202007])

# 주말 df 생성
weekend_df = pd.DataFrame(data = {'Date' : yymm, 'Weekend' : weekend}, columns = ['Date', 'Weekend'])

# 매달 공휴일 수 
holiday = [1, 3, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 3, 0, 0, 2, 1, 0, 0]

# 주말 + 공휴일
fin_holiday = [x+y for x, y in zip(weekend, holiday)]

# 휴일 df 생성
holiday_df = pd.DataFrame({'REG_YYMM' : yymm, 'Holiday' : fin_holiday})

data_final1 = pd.merge(data, holiday_df)

In [5]:
data_final1.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,Holiday
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,9
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,9
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,9
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,9
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,9


# Home Card 분류하기

- card == home : 0
- card_CCG != home_CCG : 1
- card_SIDO != home_SIDO : 2

In [6]:
# # 분류한 데이터 저장 변수 
# list_si_gu = []

# for si in range(len(data)):
#     data2 = data_final1.loc[si,:]
#     tmp = 0
    
#     # 시가 다를 때 : 2
#     if data2['CARD_SIDO_NM'] != data2['HOM_SIDO_NM']:
#         tmp=2
    
#     # 구만 다를 때 : 1
#     elif data2['CARD_CCG_NM'] != data2['HOM_CCG_NM']:
#         tmp=1
    
#     list_si_gu.append(tmp)
# data_final1['region_diff'] = list_si_gu

In [6]:
data_final1['region_diff2'] = 0
data_final1.loc[data_final1['CARD_SIDO_NM'] != data_final1['HOM_SIDO_NM'], 'region_diff2'] = 2
data_final1.loc[(data_final1['CARD_SIDO_NM'] == data_final1['HOM_SIDO_NM']) & (data_final1['CARD_CCG_NM'] != data_final1['HOM_CCG_NM']), 'region_diff2'] = 1

In [7]:
data_final1.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,Holiday,region_diff2
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,9,0
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,9,0
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,9,0
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,9,0
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,9,0


# Age 컬럼설정, 열삭제 및 이름변경

In [8]:
# AGE 컬럼 전처리
data_final1['AGE_int'] = data_final1['AGE'].str.split('s').str[0].astype(int)
data_final1.head()

# 열삭제 및 이름변경
del data_final1['AGE']
del data_final1['HOM_SIDO_NM']
del data_final1['HOM_CCG_NM']
data_final1.rename(columns={'SEX_CTGO_CD':'SEX','AGE_int':'AGE'},inplace=True)
data_final1.head(10)

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,SEX,FLC,CSTMR_CNT,AMT,CNT,Holiday,region_diff2,AGE
0,201901,강원,강릉시,건강보조식품 소매업,1,1,4,311200,4,9,0,20
1,201901,강원,강릉시,건강보조식품 소매업,1,2,7,1374500,8,9,0,30
2,201901,강원,강릉시,건강보조식품 소매업,2,2,6,818700,6,9,0,30
3,201901,강원,강릉시,건강보조식품 소매업,1,3,4,1717000,5,9,0,40
4,201901,강원,강릉시,건강보조식품 소매업,1,4,3,1047300,3,9,0,40
5,201901,강원,강릉시,건강보조식품 소매업,2,3,7,627500,7,9,0,40
6,201901,강원,강릉시,건강보조식품 소매업,1,4,21,3690830,21,9,0,50
7,201901,강원,강릉시,건강보조식품 소매업,2,4,21,3573300,20,9,0,50
8,201901,강원,강릉시,건강보조식품 소매업,1,5,19,1433500,20,9,0,60
9,201901,강원,강릉시,건강보조식품 소매업,2,5,33,7194600,40,9,0,60


### 저장

In [9]:
data_final1.to_csv('data_preprocessing_1.csv',encoding = 'utf-8-sig', index = False)